In [1]:
from graph import *
from notebook_helpers import *
import gensim

In [16]:
import igraph as ig

In [42]:
model = gensim.models.KeyedVectors.load_word2vec_format('/home/tadas/GoogleNews-vectors-negative300.bin', binary=True)

In [3]:
file = '/home/tadas/words.txt'

In [50]:
df, matrix = get_dist_matrix(model, model_type='word2vec', distance='arccos', file=file)

In [21]:
def create_graph(matrix, epsilon, df):
    matrix = np.logical_and(matrix <= epsilon, matrix != 0)
    edges = np.argwhere(matrix == True)
    
    graph = ig.Graph()
    graph.add_vertices(matrix.shape[0])
    graph.add_edges(edges)
    graph.vs['word'] = df[0]
    return graph

In [38]:
def plot(graph, filename):
    labels = list(graph.vs['word'])
    N = len(labels)
    E = [e.tuple for e in graph.es] # list of edges
    layt = graph.layout('kk') #kamada-kawai layout
    
    Xn=[layt[k][0] for k in range(N)]
    Yn=[layt[k][1] for k in range(N)]
    Xe=[]
    Ye=[]
    for e in E:
        Xe+=[layt[e[0]][0],layt[e[1]][0], None]
        Ye+=[layt[e[0]][1],layt[e[1]][1], None] 

    trace1=go.Scatter(x=Xe,
                   y=Ye,
                   mode='lines',
                   line=go.Line(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   )
    trace2=go.Scatter(x=Xn,
                   y=Yn,
                   mode='markers',
                   name='ntw',
                   marker=go.Marker(symbol='dot',
                                 size=5, 
                                 color='#6959CD',
                                 line=go.Line(color='rgb(50,50,50)', width=0.5)
                                 ),
                   text=labels,
                   hoverinfo='text'
                   )

    axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
              zeroline=False,
              showgrid=False,
              showticklabels=False,
              title='' 
              )

    width=800
    height=800
    layout=go.Layout(title= filename,  
        font= go.Font(size=12),
        showlegend=False,
        autosize=False,
        width=width,
        height=height,
        xaxis=go.XAxis(axis),
        yaxis=go.YAxis(axis),          
        margin=go.Margin(
            l=40,
            r=40,
            b=85,
            t=100,
        ),
        hovermode='closest',
        annotations=go.Annotations([
               go.Annotation(
               showarrow=False, 
                text='This igraph.Graph has the Kamada-Kawai layout',  
                xref='paper',     
                yref='paper',     
                x=0,  
                y=-0.1,  
                xanchor='left',   
                yanchor='bottom',  
                font=go.Font(
                size=14 
                )     
                )
            ]),           
        )

    data=go.Data([trace1, trace2])
    fig=go.Figure(data=data, layout=layout)
    py.plot(fig, filename=filename)

In [28]:
gr = create_graph(matrix, 3, df)

In [39]:
plot(gr, 'Word2vec_euclidean_3')

In [59]:
epsilon = 76
gr = create_graph(matrix, epsilon, df)
plot(gr, 'Word2vec_arccos_{0}'.format(epsilon))